In [1]:
import numpy as np
import pdb
# choise of the hypothesis will make problem hard 
# convergence. 

#      $_ @ _ % _ *  row experiment
# player 1 hypothesis: player 2 catches me                 or goes to [6] intrinsically
# player 2 hypothesis: player 1 wants to guide me to [6]   or goes to [0] intrinsically
size_s = 7
size_state_space = 7*7
action_list = [-1, 0, 1] # moves left or right
start_position_1 = 2 # position @
start_position_2 = 4 # position %
def dynamics(x1, x2, u1, u2):
    x1_next = x1 + action_list[u1]
    x2_next = x2 + action_list[u2]
    if x1_next > 6 or x1_next < 0:
        x1_next = x1
    if x2_next > 6 or x2_next < 0:
        x2_next = x2
    x = x1_next*7+x2_next
    return x

p1_h1 = lambda x1, x2, u1, u2 : (x1-x2)**2  + 0.1*(u2-1)**2+ 0.1*(u1-1)**2      # reward for player 2
p1_h2 = lambda x1, x2, u1, u2 : (x2-6)**2   + 0.1*(u2-1)**2+ 0.1*(u1-1)**2      # reward for player 2
p2_h1 = lambda x1, x2, u1, u2 : (x2-6)**2   + 0.1*(u1-1)**2+ 0.1*(u2-1)**2      # reward for player 1
p2_h2 = lambda x1, x2, u1, u2 : (x1)**2     + 0.1*(u1-1)**2+ 0.1*(u2-1)**2      # reward for player 1
def value_functions(r1,r2,horizon, size_s):
    NE_is_not_unique=False
    Q1  = np.zeros((horizon-1, size_s**2, 3, 3))
    Q2  = np.zeros((horizon-1, size_s**2, 3, 3))
    pi1 = np.zeros((horizon-1, size_s**2), int)
    pi2 = np.zeros((horizon-1, size_s**2), int)
    V1  = np.zeros((horizon,   size_s**2))
    V2  = np.zeros((horizon,   size_s**2))
    for x in range(size_s**2):
        x1 = int(x/size_s)
        x2 = x%size_s
        V1[-1,x] = r1(x1, x2, 1, 1)
        V2[-1,x] = r2(x1, x2, 1, 1)
    for t in np.flip(range(horizon-1)):
        for x in range(size_s**2):
            x1 = int(x/size_s)
            x2 = x%size_s
            for u1 in range(3):
                for u2 in range(3):
                    Q1[t,x,u1,u2] = r1(x1,x2,u1,u2) + V1[t+1,dynamics(x1,x2,u1,u2)] # u1 corresponds to row
                    Q2[t,x,u1,u2] = r2(x1,x2,u1,u2) + V2[t+1,dynamics(x1,x2,u1,u2)]
            pi1_tmp = np.argmin(Q1[t,x,:,:], 1) # Given player 2's row action, what's my column action?
            pi2_tmp = np.argmin(Q2[t,x,:,:], 0) # Given player 1's column action, what's my row action?
            tmp1_list = np.array(([]), dtype=np.int64)
            tmp2_list = np.array(([]), dtype=np.int64)
            for idx in range(3):
                
                if pi1_tmp[pi2_tmp[idx]] == idx:
                    tmp1_list = np.append(tmp1_list, idx)
                    tmp2_list = np.append(tmp2_list, pi2_tmp[idx])    
            if len(tmp1_list)>2:
                NE_is_not_unique=True
            elif len(tmp1_list)==0:
                import pdb; pdb.set_trace()
            pi2[t,x] = tmp1_list[0]
            pi1[t,x] = tmp2_list[0]
            
            V1[t,x] = Q1[t, x, pi1[t,x], pi2[t,x]]
            V2[t,x] = Q2[t, x, pi1[t,x], pi2[t,x]]     
    print("NE is not unique.") if NE_is_not_unique==True else None
    return V1, V2, Q1, Q2, pi1, pi2

In [2]:
horizon = 12
# player 1 hypothesis: player 2 catches me                 or goes to [6] intrinsically
# player 2 hypothesis: player 1 wants to guide me to [6]   or goes to [0] intrinsically
V1_1, V2_1, Q1_1, Q2_1, pi1_1, pi2_1 = value_functions(p2_h1,p1_h1,horizon, size_s) # ground truth
V1_2, V2_2, Q1_2, Q2_2, pi1_2, pi2_2 = value_functions(p2_h2,p1_h1,horizon, size_s) # in player 2's mind
V1_3, V2_3, Q1_3, Q2_3, pi1_3, pi2_3 = value_functions(p2_h1,p1_h2,horizon, size_s) # in player 1's mind
V1_4, V2_4, Q1_4, Q2_4, pi1_4, pi2_4 = value_functions(p2_h2,p1_h2,horizon, size_s)

In [3]:
# simulation under ground truth cost parameters
x1 = 3
x2 = 3

x_traj = np.zeros(horizon, int)
x_traj[0] = x1*7 + x2
x_list = np.zeros((horizon,2), int)
x_list[0,0] = x1
x_list[0,1] = x2
for t in range(horizon-1):
    action1 = pi1_1[t,int(x_traj[t])]
    action2 = pi2_1[t,int(x_traj[t])]
    x_traj[t+1] = dynamics(x_list[t,0],x_list[t,1], action1, action2)
    x_list[t+1,0] = int(x_traj[t+1]/size_s)
    x_list[t+1,1] = x_traj[t+1]%size_s
print("sampled trajectory:")
print(x_list.transpose())

sampled trajectory:
[[3 4 5 6 6 6 6 6 6 6 6 6]
 [3 4 5 6 6 6 6 6 6 6 6 6]]


In [4]:
# in player 2's mind: I want to catch player 1, player 1 wants to go to [0]
x1=3
x2=3

x_traj = np.zeros(horizon, int)
x_traj[0] = x1*7 + x2
x_list = np.zeros((horizon,2), int)
x_list[0,0] = x1
x_list[0,1] = x2
for t in range(horizon-1):
    action1 = pi1_2[t,int(x_traj[t])]
    action2 = pi2_2[t,int(x_traj[t])]
    x_traj[t+1] = dynamics(x_list[t,0],x_list[t,1], action1, action2)
    x_list[t+1,0] = int(x_traj[t+1]/size_s)
    x_list[t+1,1] = x_traj[t+1]%size_s
print("sampled trajectory:")
print(x_list.transpose())

sampled trajectory:
[[3 2 1 0 0 0 0 0 0 0 0 0]
 [3 2 1 0 0 0 0 0 0 0 0 0]]


In [5]:
# in player 1's mind: I want to guide player 2 to [6], player 2 wants to go to [6]
x1 = 3
x2 = 3

x_traj = np.zeros(horizon, int)
x_traj[0] = x1*7 + x2
x_list = np.zeros((horizon,2), int)
x_list[0,0] = x1
x_list[0,1] = x2
for t in range(horizon-1):
    action1 = pi1_3[t,int(x_traj[t])]
    action2 = pi2_3[t,int(x_traj[t])]
    x_traj[t+1] = dynamics(x_list[t,0],x_list[t,1], action1, action2)
    x_list[t+1,0] = int(x_traj[t+1]/size_s)
    x_list[t+1,1] = x_traj[t+1]%size_s
print("sampled trajectory:")
print(x_list.transpose())

sampled trajectory:
[[3 3 3 3 3 3 3 3 3 3 3 3]
 [3 4 5 6 6 6 6 6 6 6 6 6]]


In [6]:
# simulation never happens
x1 = 3
x2 = 3

x_traj = np.zeros(horizon, int)
x_traj[0] = x1*7 + x2
x_list = np.zeros((horizon,2), int)
x_list[0,0] = x1
x_list[0,1] = x2
for t in range(horizon-1):
    action1 = pi1_4[t,int(x_traj[t])]
    action2 = pi2_4[t,int(x_traj[t])]
    x_traj[t+1] = dynamics(x_list[t,0],x_list[t,1], action1, action2)
    x_list[t+1,0] = int(x_traj[t+1]/size_s)
    x_list[t+1,1] = x_traj[t+1]%size_s
print("sampled trajectory:")
print(x_list.transpose())

sampled trajectory:
[[3 3 3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3 3 3]]


In [7]:
# Define belief update rule:
# player 1's task policy: pi1_3,  observation: u2=pi2_2(t,x),  update belief according to: pi2_3
# player 2's task policy: pi2_2,  observation: u1=pi1_3(t,x),  update belief according to: pi1_2
x1=3
x2=3
def belief_update_p1(prior, u2, t, x):
    if prior == 1:
        if u2!=pi2_3[t,x]:
            posterior = 0
        else:
            posterior = 1
    else:
        if u2!=pi2_1[t,x]:
            posterior = 1
        else:
            posterior = 0
    return posterior
def belief_update_p2(prior, u1, t, x):
    if prior == 1:
        if u1!=pi1_2[t,x]:
            posterior = 0
        else:
            posterior = 1
    else:
        if u1!=pi1_1[t,x]:
            posterior = 1
        else:
            posterior = 0
    return posterior

In [69]:
# Roll-out trajectory
x1=1                                     # initial state of player 1
x2=4                                     # initial state of player 2
x_traj_list = np.zeros((horizon,2), int) # state trajectory, understandable by human
x_combined = np.zeros(horizon, int)      # state trajectory, understandable by computer
x_traj_list[0,0] = x1                    # initial condition of player 1
x_traj_list[0,1] = x2                    # initial condition of player 2
x_combined[0] = x1*7+x2                  
b1_list = np.zeros(horizon, int)         # player 1's belief 
b2_list = np.zeros(horizon, int)         # player 2's belief
b1_list[0] = 1                           # player 1's initial belief
b2_list[0] = 0                           # player 2's initial belief
u1_list=np.zeros(horizon-1, int)         # player 1's control trajectory
u2_list=np.zeros(horizon-1, int)         # player 2's control trajectory

for t in range(horizon-1):
    policy_of_player1 = pi1_1 if b1_list[t] == 0 else pi1_3
    policy_of_player2 = pi2_1 if b2_list[t] == 0 else pi2_2
    u1_list[t] = policy_of_player1[t,x_combined[t]]
    u2_list[t] = policy_of_player2[t,x_combined[t]]
    
    x_combined[t+1] = dynamics(x_traj_list[t,0], x_traj_list[t,1],u1_list[t],u2_list[t])
    x_traj_list[t+1,0] = int(x_combined[t+1]/size_s)
    x_traj_list[t+1,1] = x_combined[t+1]%size_s
    
    b1_list[t+1] = belief_update_p1(b1_list[t], u2_list[t], t,x_combined[t])
    b2_list[t+1] = belief_update_p2(b2_list[t], u1_list[t], t,x_combined[t])

# player 1's belief converges as time goes. Initial belief: b1 = 1 \in {0,1}
print("player 1's belief evolution:")
print(b1_list)
# player 2's belief converges as time goes. Initial belief: b2 = 0 \in {0,1}
print("player 2's belief evolution:")
print(b2_list)
# simulated state trajectories of player 1 and player 2. Initial state x1[0] = 3, x2[0] = 3.
print("roll-out trajectory of two players:")
print(x_traj_list.transpose())

player 1's belief evolution:
[1 1 0 0 0 0 0 0 0 0 0 0]
player 2's belief evolution:
[0 1 0 0 0 0 0 0 0 0 0 0]
roll-out trajectory of two players:
[[1 1 1 2 3 4 5 6 6 6 6 6]
 [4 5 4 5 6 6 6 6 6 6 6 6]]


In [118]:
# build the value iteration for belief update
# 1. Reward
# 2. Belief Dynamics

size_s = 7
n_belief = 2*2
size_state_space = 7*7 * n_belief
action_list = [-1, 0, 1] # moves left or right
start_position_1 = 2 # position @
start_position_2 = 4 # position %
def new_dynamics(x1, x2, u1, u2, b1, b2, v1, v2,t):
    x_current = x1*7+x2
    x1_next = x1 + action_list[u1] + action_list[v1]
    x2_next = x2 + action_list[u2] + action_list[v2]
    if x1_next > 6 or x1_next < 0:
        x1_next = x1
    if x2_next > 6 or x2_next < 0:
        x2_next = x2
    x = x1_next*7+x2_next
    b1_next = belief_update_p1(b1, u2, t, x_current)
    b2_next = belief_update_p2(b2, u1, t, x_current)
    return x + b1_next*size_s**2 + b2_next*size_s**2*2

def policy_under_belief_player1(t, x, b1):
    u1 = pi1_1[t,x] if b1 == 0 else pi1_3[t,x]
    return u1
def policy_under_belief_player2(t, x, b2):
    u2 = pi2_1[t,x] if b2 == 0 else pi2_2[t,x]
    return u2

def bvalue_functions(horizon, size_s, exploration_cost):
    NE_is_not_unique=False
    Q1  = np.zeros((horizon-1, size_s**2*2*2, 3, 3))
    Q2  = np.zeros((horizon-1, size_s**2*2*2, 3, 3))
    pi1 = np.zeros((horizon-1, size_s**2*2*2), int)
    pi2 = np.zeros((horizon-1, size_s**2*2*2), int)
    V1  = np.zeros((horizon,   size_s**2*2*2))
    V2  = np.zeros((horizon,   size_s**2*2*2))
    r1 = p2_h1
    r2 = p1_h1
    for x in range(size_s**2):
        x1 = int(x/size_s)
        x2 = x%size_s
        V1[-1,x] = r1(x1, x2, 1, 1)
        V2[-1,x] = r2(x1, x2, 1, 1)
    for t in np.flip(range(horizon-1)):
        for x in range(size_s**2):
            x1 = int(x/size_s)
            x2 = x%size_s
            for b1 in range(2):
                for b2 in range(2):
                    u1 = policy_under_belief_player1(t, x, b1)
                    u2 = policy_under_belief_player2(t, x, b2)
                    s = x + b1*size_s**2 + b2*2*size_s**2
                    
                    for v1 in range(3):
                        for v2 in range(3):
                            Q1[t,s,v1,v2] = r1(x1,x2,u1,u2) + exploration_cost*(v1-1)**2 + exploration_cost*(v2-1)**2 + V1[t+1,new_dynamics(x1,x2,u1,u2,b1,b2,v1,v2,t)] 
                            # u1 corresponds to row
                            Q2[t,s,v1,v2] = r2(x1,x2,u1,u2) + exploration_cost*(v1-1)**2 + exploration_cost*(v2-1)**2 + V2[t+1,new_dynamics(x1,x2,u1,u2,b1,b2,v1,v2,t)]
                    
                    # ----------------- 
                    tmp1_list = {0:np.array([], dtype=np.int64), 1:np.array([], dtype=np.int64), 
                                 2:np.array([], dtype=np.int64)}
                    tmp2_list = {0:np.array([], dtype=np.int64), 1:np.array([], dtype=np.int64),
                                 2:np.array([], dtype=np.int64)}
#                     import pdb; pdb.set_trace()
                    # for player 1:
                    for idx2 in range(3):
                        for idx1 in range(3):
                            if Q1[t,s,idx1,idx2] == np.min(Q1[t,s,idx2,:]):
                                tmp1_list[idx2] = np.append(tmp1_list[idx2], idx1)
#                                 import pdb; pdb.set_trace()
                    # for player 2:
                    for idx1 in range(3):
                        for idx2 in range(3):
                            if Q2[t,s,idx1,idx2] == np.min(Q2[t,s,:,idx1]):
                                tmp2_list[idx1] = np.append(tmp2_list[idx1], idx2)
                    # find NE
                    policy1_list = np.array([], dtype=np.int64)
                    policy2_list = np.array([], dtype=np.int64)
                    for idx1 in range(3):
                        for idx2 in range(3):
                            if idx1 in tmp1_list[idx2] and idx2 in tmp2_list[idx1]:
                                policy1_list = np.append(policy1_list, idx1)
                                policy2_list = np.append(policy2_list, idx2)

                    if len(policy1_list)>2:
                        NE_is_not_unique=True
                    elif len(policy1_list)==0:                        
                        import pdb; pdb.set_trace()
                    pi2[t,s] = policy1_list[0]
                    pi1[t,s] = policy2_list[0]

                    V1[t,s] = Q1[t, s, pi1[t,s], pi2[t,s]]
                    V2[t,s] = Q2[t, s, pi1[t,s], pi2[t,s]]     
    print("NE is not unique.") if NE_is_not_unique==True else None
    return V1, V2, Q1, Q2, pi1, pi2


"""
         action 1
         --------
         |x x x
action 2 |x x x
         |x x x
         
"""

'\n         action 1\n         --------\n         |x x x\naction 2 |x x x\n         |x x x\n         \n'

In [119]:
def compact_coordinate_to_readable_format(s):
    x = s%(size_s**2)
    x1 = int(x/size_s)
    x2 = x%size_s 
    if int(s/size_s**2) == 3:
        b1 = 1
        b2 = 1
    else:
        b2 = int(s/(2*size_s**2))
        b1 = int((s-size_s**2*2*b2)/size_s**2)
    return x1, x2, b1, b2
def readable_format_to_compact_coordinate(x1,x2,b1,b2):
    return x1*size_s + x2 + b1*size_s**2 + b2*size_s**2*2


In [120]:
b1=0
b2=1
x1 = 1
x2 = 4
x = x1*size_s+x2
compact_coordinate_to_readable_format(readable_format_to_compact_coordinate(x1,x2,b1,b2))

(1, 4, 0, 1)

In [117]:
horizon = 5
# player 1 hypothesis: player 2 catches me                 or goes to [6] intrinsically
# player 2 hypothesis: player 1 wants to guide me to [6]   or goes to [0] intrinsically
bV1, bV2, bQ1, bQ2, bpi1, bpi2 = bvalue_functions(horizon, size_s, 2.2)

> <ipython-input-60-ca88be4006dd>(92)bvalue_functions()
     90                     elif len(policy1_list)==0:
     91                         import pdb; pdb.set_trace()
---> 92                     pi2[t,s] = policy1_list[0]
     93                     pi1[t,s] = policy2_list[0]
     94 

ipdb> t
0
ipdb> Q1[t,s]
array([[120.3, 134.1, 120.3],
       [100.3, 162.9, 100.3],
       [145.4, 169.5, 145.4]])
ipdb> Q2[t,s]
array([[10.3, 14.1, 10.3],
       [24.3, 22.9, 24.3],
       [21.4, 77.5, 21.4]])
ipdb> tmp1_list
{0: array([1]), 1: array([0]), 2: array([1])}
ipdb> tmp2_list
{0: array([0, 2]), 1: array([1]), 2: array([0, 2])}
ipdb> exit


BdbQuit: 

In [116]:
# simulate the system
# Roll-out trajectory
x1=3                                     # initial state of player 1
x2=4                                     # initial state of player 2
x_traj_list = np.zeros((horizon,2), int) # state trajectory, understandable by human
x_combined = np.zeros(horizon, int)      # state trajectory, understandable by computer
x_traj_list[0,0] = x1                    # initial condition of player 1
x_traj_list[0,1] = x2                    # initial condition of player 2
x_combined[0] = x1*7+x2                  
b1_list = np.zeros(horizon, int)         # player 1's belief 
b2_list = np.zeros(horizon, int)         # player 2's belief
b1_list[0] = 0                           # player 1's initial belief
b2_list[0] = 1                           # player 2's initial belief
u1_list=np.zeros(horizon-1, int)         # player 1's control trajectory
u2_list=np.zeros(horizon-1, int)         # player 2's control trajectory
v1_list=np.zeros(horizon-1, int)         # player 1's exploration action trajectory
v2_list=np.zeros(horizon-1, int)         # player 2's exploration action trajectory


for t in range(horizon-1):
    policy_of_player1 = pi1_1 if b1_list[t] == 0 else pi1_3
    policy_of_player2 = pi2_1 if b2_list[t] == 0 else pi2_2
    s = readable_format_to_compact_coordinate(x_traj_list[t,0],x_traj_list[t,1],b1_list[t],b2_list[t])
    v1_list[t] = bpi1[t,s]
    v2_list[t] = bpi2[t,s]
    u1_list[t] = policy_of_player1[t,x_combined[t]]
    u2_list[t] = policy_of_player2[t,x_combined[t]]
    
    new_s=new_dynamics(x_traj_list[t,0],x_traj_list[t,1],u1_list[t],u2_list[t],
                       b1_list[t],b2_list[t],v1_list[t],v2_list[t],t)
    x_traj_list[t+1,0], x_traj_list[t+1,1], b1_list[t+1], b2_list[t+1] = compact_coordinate_to_readable_format(
                                                                                                        new_s)
    x_combined[t+1] = x_traj_list[t+1,0]*size_s + x_traj_list[t+1,1]

# player 1's belief converges as time goes. Initial belief: b1 = 1 \in {0,1}
print("player 1's belief evolution:")
print(b1_list)
# player 2's belief converges as time goes. Initial belief: b2 = 0 \in {0,1}
print("player 2's belief evolution:")
print(b2_list)
# simulated state trajectories of player 1 and player 2. Initial state x1[0] = 3, x2[0] = 3.
print("roll-out trajectory with belief update:")
print(x_traj_list.transpose())

player 1's belief evolution:
[0 1 1 1]
player 2's belief evolution:
[1 0 0 1]
roll-out trajectory with belief update:
[[3 3 3 3]
 [4 2 3 4]]


In [106]:
v1_list

array([0, 1, 1])

In [107]:
v2_list

array([0, 1, 1])

In [17]:
print(b1_list)

[0 1 0 1 1 0]


In [18]:
print(b2_list)

[1 1 1 0 1 0]


In [19]:
u1_list

array([2, 1, 2, 1, 1])